In [1]:
library(ggplot2)
library(dplyr)
library(tidyr)
library(magrittr)
library(zoo)
library(purrr)
library(scales)
library(stringr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘magrittr’

The following object is masked from ‘package:tidyr’:

    extract



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘magrittr’

The following object is masked from ‘package:tidyr’:

    extract




Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘purrr’

The following object is masked from ‘package:magrittr’:

    set_names


Attaching package: ‘scales’

The following object is masked from ‘package:purrr’:

    discard



Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘purrr’

The following object is masked from ‘package:magrittr’:

    set_names


Attaching package: ‘scales’

The following object is masked from ‘package:purrr’:

    discard



Cargamos los datos:

In [2]:
cat = readRDS('../02_Variables/cat.rds')
num = readRDS('../02_Variables/num.rds')

Unimos ambas dos variables de nuevo en un dataframe.

In [3]:
df = bind_cols(cat, num)

In [4]:
df %>% colnames()

 [1] "date"         "store_id"     "item_id"      "d"            "year"         "month"        "wday"         "weekday"     
 [9] "event_name_1" "event_type_1" "wm_yr_wk"     "ventas"       "sell_price"  

Vamos a crear primero variables sintéticas que nos ayudarán con la fase de forecasting. Después haremos transformaciones sobre las propias variables para que sean aptas para el modelo de Deep Learning.

Identificamos los momentos donde haya podido haber rotura de stock,, es decir, ventas cero durante un periodo.

Definimos una función para identificar la rotura de stock, donde primeramente se identifiquen los días donde no haya habido ventas con un flag de 1/0, después a través del parámetro n que establecemos en 5 días como primera aproximación, definimos la rotura de stock, cuando en 5 días no ha habido ventas, para hacer conteo de esto, hacemos una suma móvil donde capturemos la información de esos cinco días (ó n), después confirmamos sí hay (ó no), rotura de stock.

Creamos también funciones de ventanas móviles (media, mínimo y máximo).

In [5]:
rotura_stock = function(ventas, n = 5) {
 
  cero_ventas = ifelse(ventas == 0, 1, 0)
  
  num_ceros = zoo::rollsum(cero_ventas, k = n, fill = 0, align = "right")
  
  rotura_stock = ifelse(num_ceros == n, 1, 0)
  
  return(rotura_stock)
}

In [6]:
crear_lags = function(df, variable, num_lags = 7) {
  
  variable_name = deparse(substitute(variable))
  
  lags = data.frame(matrix(ncol = num_lags, nrow = nrow(df)))
  colnames(lags) = paste0(variable_name, "_lag_", 1:num_lags)
 
  for (elemento in 1:num_lags) {
    lag_name = colnames(lags)[elemento]
    lags[[lag_name]] = dplyr::lag(df[[variable_name]], n = elemento)
  }
  
  return(lags)
}

In [7]:
min_movil = function(df, variable, num_periodos = 7) {
  
  variable_name = deparse(substitute(variable))
  
  minm = data.frame(matrix(ncol = num_periodos - 1, nrow = nrow(df)))
  colnames(minm) = paste0(variable_name, "_min_", 2:num_periodos)
 
  for (elemento in 2:num_periodos) {
    min_name = colnames(minm)[elemento - 1]
    minm[[min_name]] = rollapply(df[[variable_name]], width = elemento, FUN = min, fill = NA, align = 'right')
  }
  
  return(minm)
}

In [8]:
max_movil = function(df, variable, num_periodos = 7) {
  
  variable_name = deparse(substitute(variable))
  
  maxm = data.frame(matrix(ncol = num_periodos - 1, nrow = nrow(df)))
  colnames(maxm) = paste0(variable_name, "_max_", 2:num_periodos)
 
  for (elemento in 2:num_periodos) {
    max_name = colnames(maxm)[elemento - 1]
    maxm[[max_name]] = rollapply(df[[variable_name]], width = elemento, FUN = max, fill = NA, align = 'right')
  }
  
  return(maxm)
}

In [9]:
media_movil = function(df, variable, num_periodos = 7) {
  
  variable_name = deparse(substitute(variable))
  
  media = data.frame(matrix(ncol = num_periodos - 1, nrow = nrow(df)))
  colnames(media) = paste0(variable_name, "_media_", 2:num_periodos)
 
  for (elemento in 2:num_periodos) {
    media_name = colnames(media)[elemento - 1]
    media[[media_name]] = rollapply(df[[variable_name]], width = elemento, FUN = mean, fill = NA, align = 'right')
  }
  
  return(media)
}

Vamos a segmentar df por tienda y producto, previamente a crear las variables.

In [10]:
nombre_tiendas = df$store_id %>% unique()

In [11]:
nombre_productos = df$item_id %>% unique()

In [12]:
lista_df_tienda_producto = list()

In [13]:
for (ntienda in nombre_tiendas){

  df_tienda = df %>% filter(store_id == ntienda)

  for (nproducto in nombre_productos){

    df_producto = df_tienda %>% filter(item_id == nproducto)

    nombre = paste0("df", "_" , ntienda , "_" , nproducto)

    lista_df_tienda_producto[[nombre]] = df_producto


  }


}

Comprobamos la creación.

In [14]:
lista_df_tienda_producto %>% names()

 [1] "df_CA_3_FOODS_3_090" "df_CA_3_FOODS_3_120" "df_CA_3_FOODS_3_202" "df_CA_3_FOODS_3_252" "df_CA_3_FOODS_3_288" "df_CA_3_FOODS_3_329"
 [7] "df_CA_3_FOODS_3_555" "df_CA_3_FOODS_3_586" "df_CA_3_FOODS_3_587" "df_CA_3_FOODS_3_714" "df_CA_4_FOODS_3_090" "df_CA_4_FOODS_3_120"
[13] "df_CA_4_FOODS_3_202" "df_CA_4_FOODS_3_252" "df_CA_4_FOODS_3_288" "df_CA_4_FOODS_3_329" "df_CA_4_FOODS_3_555" "df_CA_4_FOODS_3_586"
[19] "df_CA_4_FOODS_3_587" "df_CA_4_FOODS_3_714"

Creamos las variables sintéticas en cada uno de los dataframes, procesamos, unimos, eliminamos nulos creados y duplicados y volvemos a guardar.

In [15]:
lista_df_tienda_producto_2 = list()

for (dataframe in names(lista_df_tienda_producto)){

  nombre_dataframe = dataframe

  df_bucle = lista_df_tienda_producto[[nombre_dataframe]]

  df_bucle = df_bucle %>% arrange(store_id, item_id, date) %>%  group_by(store_id, item_id) %>% mutate(
    rotura_stock_3 = rotura_stock(ventas, 3),
    rotura_stock_7 = rotura_stock(ventas, 7) ) %>% ungroup()
  
  lags_sell_price = crear_lags(df_bucle, sell_price, 7)
  lags_ventas = crear_lags(df_bucle, ventas, 7)
  lags_rotura_stock_3 = crear_lags(df_bucle, rotura_stock_3, 1)
  lags_rotura_stock_7 = crear_lags(df_bucle, rotura_stock_7, 1)

  media_movil_df = media_movil(df_bucle, ventas, 7)
  min_movil_df = min_movil(df_bucle, ventas, 7)
  max_movil_df = max_movil(df_bucle, ventas, 7)

  df_final = bind_cols(df_bucle,
    lags_sell_price,
    lags_ventas,
    lags_rotura_stock_3,
    lags_rotura_stock_7,
    media_movil_df,
    min_movil_df,
    max_movil_df)
  
  df_final = df_final[, !duplicated(names(df_final))]

  df_final = df_final %>% drop_na()

  variables_eliminar = c('d','wm_yr_wk','sell_price','rotura_stock_3','rotura_stock_7')

  df_final = df_final %>% select(-all_of(variables_eliminar))

  lista_df_tienda_producto_2[[nombre_dataframe]] = df_final


}

Previamente a hacer Min Max Scaling, guardamos los valores máximos y mínimos para poder revertirlo, una vez hecho el forecasting más adelante.

In [16]:
lista_min_max_ventas = list()

for (dataframe in names(lista_df_tienda_producto_2)){

  nombre_dataframe = dataframe

  df_bucle = lista_df_tienda_producto_2[[nombre_dataframe]]

  min_bucle = df_bucle$ventas %>% min()
  max_bucle = df_bucle$ventas %>% max()

  nombre_min = paste0("min","_", str_remove(nombre_dataframe,"df_"))
  nombre_max = paste0("max","_", str_remove(nombre_dataframe,"df_"))

  lista_min_max_ventas[[nombre_min]] = min_bucle
  lista_min_max_ventas[[nombre_max]] = max_bucle
  
  
}

Guardamos también los valores reales de ventas en el histórico para la fase finald de visualización tras el forecasting.

In [17]:
saveRDS(lista_df_tienda_producto_2, file = "../02_Variables/lista_df_valores_reales.rds")

A continuación, procedemos a aplicar One Hot Encoding y Min Max Scaling a los dataframes generados.

In [18]:
lista_df_tienda_producto_3 = list()

for (dataframe in names(lista_df_tienda_producto_2)){

  nombre_dataframe = dataframe

  df_bucle = lista_df_tienda_producto_2[[nombre_dataframe]]

  num = df_bucle %>% select(where(is.numeric))
  cat = df_bucle %>% select_if(~ !is.numeric(.))

  var_ohe = c('year', 'month', 'wday', 'weekday', 'event_name_1', 'event_type_1')

  var_ohe_matrix = paste("~", paste(var_ohe, collapse = " + "), "- 1")
  cat_ohe_matrix = model.matrix(as.formula(var_ohe_matrix), data = cat)

  cat_ohe = as.data.frame(cat_ohe_matrix)

  df_var = df_bucle %>% select(date, store_id, item_id)

  df_mm = cbind(num, cat_ohe)

  df_mm = df_mm %>% mutate(across(everything(), ~ rescale(.x, to = c(-1, 1))))

  df_td = cbind(df_var, df_mm)

  lista_df_tienda_producto_3[[nombre_dataframe]] = df_td

  
}

Guardamos los resultados, antes del proceso de preselección de variables.

In [19]:
saveRDS(lista_df_tienda_producto_3, file = "../02_Variables/lista_df_preseleccion_var.rds")
saveRDS(lista_min_max_ventas, file = "../02_Variables/lista_min_max_ventas.rds")